# **Extracting, transforming and loading data**


In [1]:
import numpy as np
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from hoopshunch import database_builder
from hoopshunch.database import Database
from hoopshunch import templates
import warnings

warnings.filterwarnings("ignore")

### **Extracting data**


In [2]:
# database_builder.build_database("../data/database/nba.db", 2012, 2012, "replace", 1)
# database_builder.build_database("nba.db", 2022, 2022, "append", 3)

In [3]:
database = Database("../data/database/nba.db")
games = database.create_model_stats()

In [4]:
games.head()

,SEASON,HOME_TEAM_ID,AWAY_TEAM_ID,GAME_DATE,MATCHUP,HOME_SPREAD,AWAY_SPREAD,OVER_UNDER,HOME_MONEYLINE,AWAY_MONEYLINE,...,AWAY_P5_STL,AWAY_P5_TOV,AWAY_P5_FGA,AWAY_P5_FTA,AWAY_P5_FG3A,AWAY_P5_PF,HOME_PTS,AWAY_PTS,GAME_ID,HOME_WL
0,2012,1610612739,1610612764,2012-10-30,CLE vs. WAS,-5.5,5.5,191.0,-250.0,210.0,...,0,1,13,1,9,1,94,84,0021200001,W
1,2012,1610612747,1610612742,2012-10-30,LAL vs. DAL,-8.5,8.5,187.0,-500.0,400.0,...,1,0,10,2,0,4,91,99,0021200003,L
2,2012,1610612748,1610612738,2012-10-30,MIA vs. BOS,-8.0,8.0,184.0,-280.0,240.0,...,1,3,7,4,3,3,120,107,0021200002,W
3,2012,1610612756,1610612744,2012-10-31,PHX vs. GSW,-3.5,3.5,201.5,-140.0,120.0,...,2,4,14,3,6,3,85,87,0021200010,L
4,2012,1610612755,1610612743,2012-10-31,PHI vs. DEN,2.0,-2.0,201.0,110.0,-130.0,...,1,1,3,4,0,3,84,75,0021200005,W


### **Transforming data**


- Null Values


In [5]:
def see_null_values(dataframe):
    for row in dataframe.columns:
        num_null_values = dataframe[row].isnull().sum()
        if num_null_values > 0:
            print(f"Row {row} have {num_null_values} null values")


see_null_values(games)

Row SEASON_HOME_GP have 200 null values
Row SEASON_HOME_W have 200 null values
Row SEASON_HOME_L have 200 null values
Row SEASON_HOME_W_PCT have 200 null values
Row SEASON_HOME_PTS have 200 null values
Row SEASON_HOME_W_RANK have 200 null values
Row SEASON_HOME_L_RANK have 200 null values
Row SEASON_HOME_W_PCT_RANK have 200 null values
Row SEASON_HOME_PTS_RANK have 200 null values
Row SEASON_AWAY_GP have 200 null values
Row SEASON_AWAY_W have 200 null values
Row SEASON_AWAY_L have 200 null values
Row SEASON_AWAY_W_PCT have 200 null values
Row SEASON_AWAY_PTS have 200 null values
Row SEASON_AWAY_W_RANK have 200 null values
Row SEASON_AWAY_L_RANK have 200 null values
Row SEASON_AWAY_W_PCT_RANK have 200 null values
Row SEASON_AWAY_PTS_RANK have 200 null values
Row LAST5_HOME_GP have 157 null values
Row LAST5_HOME_W have 157 null values
Row LAST5_HOME_L have 157 null values
Row LAST5_HOME_W_PCT have 157 null values
Row LAST5_HOME_MIN have 157 null values
Row LAST5_HOME_FGM have 157 null va

In [6]:
games = games.dropna(
    subset=["SEASON_HOME_GP", "SEASON_AWAY_GP", "LAST5_HOME_GP", "LAST5_AWAY_GP"]
)
see_null_values(games)

- Duplicates


In [7]:
duplicates = games[games.duplicated()]
print(duplicates.count())


SEASON          0
HOME_TEAM_ID    0
AWAY_TEAM_ID    0
GAME_DATE       0
MATCHUP         0
               ..
AWAY_P5_PF      0
HOME_PTS        0
AWAY_PTS        0
GAME_ID         0
HOME_WL         0
Length: 233, dtype: int64


- API data download issues


In [8]:
def fix_errors(games):
    num_occurrences_h1 = (games["INJURED_STAR1_HOME"] == 99).sum()
    num_occurrences_h2 = (games["INJURED_STAR2_HOME"] == 99).sum()
    num_occurrences_h3 = (games["INJURED_STAR3_HOME"] == 99).sum()
    num_occurrences_a1 = (games["INJURED_STAR1_AWAY"] == 99).sum()
    num_occurrences_a2 = (games["INJURED_STAR2_AWAY"] == 99).sum()
    num_occurrences_a3 = (games["INJURED_STAR3_AWAY"] == 99).sum()

    print(f"Issues in column INJURED_STAR1_HOME: {num_occurrences_h1}")
    print(f"Issues in column INJURED_STAR2_HOME: {num_occurrences_h2}")
    print(f"Issues in column INJURED_STAR3_HOME: {num_occurrences_h3}")
    print(f"Issues in column INJURED_STAR1_AWAY: {num_occurrences_a1}")
    print(f"Issues in column INJURED_STAR2_AWAY: {num_occurrences_a2}")
    print(f"Issues in column INJURED_STAR3_AWAY: {num_occurrences_a3}")

    games["INJURED_STAR1_HOME"] = games["INJURED_STAR1_HOME"].replace(99, 0)
    games["INJURED_STAR2_HOME"] = games["INJURED_STAR2_HOME"].replace(99, 0)
    games["INJURED_STAR3_HOME"] = games["INJURED_STAR3_HOME"].replace(99, 0)
    games["INJURED_STAR1_AWAY"] = games["INJURED_STAR1_AWAY"].replace(99, 0)
    games["INJURED_STAR2_AWAY"] = games["INJURED_STAR2_AWAY"].replace(99, 0)
    games["INJURED_STAR3_AWAY"] = games["INJURED_STAR3_AWAY"].replace(99, 0)


fix_errors(games)

Issues in column INJURED_STAR1_HOME: 0
Issues in column INJURED_STAR2_HOME: 5
Issues in column INJURED_STAR3_HOME: 2
Issues in column INJURED_STAR1_AWAY: 0
Issues in column INJURED_STAR2_AWAY: 3
Issues in column INJURED_STAR3_AWAY: 4


In [9]:
fix_errors(games)


Issues in column INJURED_STAR1_HOME: 0
Issues in column INJURED_STAR2_HOME: 0
Issues in column INJURED_STAR3_HOME: 0
Issues in column INJURED_STAR1_AWAY: 0
Issues in column INJURED_STAR2_AWAY: 0
Issues in column INJURED_STAR3_AWAY: 0


#### Improving some columns and creating new ones

- Shortening the team id


In [10]:
print(games["HOME_TEAM_ID"].unique())

[1610612752 1610612760 1610612747 1610612761 1610612753 1610612746
 1610612759 1610612751 1610612755 1610612763 1610612742 1610612748
 1610612758 1610612743 1610612741 1610612744 1610612745 1610612738
 1610612749 1610612737 1610612740 1610612750 1610612766 1610612762
 1610612757 1610612756 1610612764 1610612754 1610612765 1610612739]


In [11]:
games["HOME_TEAM_ID"] = games["HOME_TEAM_ID"].apply(lambda x: x % 100)
games["AWAY_TEAM_ID"] = games["AWAY_TEAM_ID"].apply(lambda x: x % 100)
print(games["AWAY_TEAM_ID"].unique())

[55 37 65 50 56 39 54 52 62 57 44 61 53 43 64 63 59 51 47 60 46 45 48 66
 42 49 38 58 41 40]


- Getting the probability of each team from the spread


In [12]:
k = 0.1
games["HOME_SPREAD_PROB"] = 1 / (1 + np.exp(-k * games["HOME_SPREAD"]))
games["AWAY_SPREAD_PROB"] = 1 / (1 + np.exp(-k * games["AWAY_SPREAD"]))

- Creating columns won or lost spread


In [13]:
games["HOME_SPREAD_WL"] = 0
games.loc[
    games["HOME_PTS"] - games["AWAY_PTS"] > games["HOME_SPREAD"], "HOME_SPREAD_WL"
] = 1

# Crear columna AWAY_SPREAD_WL
games["AWAY_SPREAD_WL"] = 0
games.loc[
    games["AWAY_PTS"] - games["HOME_PTS"] > games["AWAY_SPREAD"], "AWAY_SPREAD_WL"
] = 1


- Moneyline to decimal


In [14]:
def moneyline_to_decimal(x):
    if x > 0:
        return (x / 100) + 1
    elif x < 0:
        return (100 / abs(x)) + 1
    else:
        return 1.0


games["HOME_MONEYLINE_DECIMAL"] = games["HOME_MONEYLINE"].apply(moneyline_to_decimal)
games["AWAY_MONEYLINE_DECIMAL"] = games["AWAY_MONEYLINE"].apply(moneyline_to_decimal)

- Getting the level of fatigue from the rest days


In [15]:
def convert_rest_days_to_num(rest_days):
    if rest_days == "3+" or rest_days.endswith("2"):
        return 0
    elif rest_days == "1":
        return 1 / 5
    elif rest_days == "3IN4":
        return 2 / 5
    elif rest_days == "B2B":
        return 3 / 5
    elif rest_days == "3IN4-B2B":
        return 4 / 5
    elif rest_days == "4IN5-B2B":
        return 5 / 5
    else:
        raise ValueError(f"Unknown rest days: {rest_days}")


games["HOME_FATIGUE_LEVEL"] = games["HOME_REST_DAYS"].apply(convert_rest_days_to_num)
games["AWAY_FATIGUE_LEVEL"] = games["AWAY_REST_DAYS"].apply(convert_rest_days_to_num)

- Getting the total points and whether it was over or under


In [16]:
games["TOTAL_PTS"] = games["HOME_PTS"] + games["AWAY_PTS"]


def over_under_wl(row):
    if row["TOTAL_PTS"] >= row["OVER_UNDER"]:
        return 1
    else:
        return 0


games["OVER_UNDER_WL"] = games.apply(over_under_wl, axis=1)

In [17]:
games["HOME_WL"] = games["HOME_WL"].replace({"W": 1, "L": 0})

- Sorting and formatting


In [18]:
columns_list = games.columns.tolist()
# for column in columns_list:
#     print(column)

ordened_columns, type_columns = templates.columns_to_export()
games = games.reindex(columns=ordened_columns).astype(type_columns)

float_cols = games.select_dtypes(include=["float"]).columns
games[float_cols] = games[float_cols].round(2)

### **Loading data to DB and save as CSV**


In [19]:
database.upload_model_stats(games)
games.to_csv("../data/processed/NBA.csv")
games.head()

,SEASON,GAME_ID,GAME_DATE,HOME_TEAM_ID,AWAY_TEAM_ID,MATCHUP,HOME_SPREAD,AWAY_SPREAD,HOME_SPREAD_PROB,AWAY_SPREAD_PROB,...,AWAY_P5_AST,AWAY_P5_DREB,AWAY_P5_OREB,AWAY_P5_BLK,AWAY_P5_STL,AWAY_P5_TOV,AWAY_P5_FGA,AWAY_P5_FTA,AWAY_P5_FG3A,AWAY_P5_PF
35,2012,0021200037,2012-11-04,52,55,NYK vs. PHI,-4.5,4.5,0.39,0.61,...,0,2,0,0,0,1,10,0,3,3
36,2012,0021200040,2012-11-04,60,37,OKC vs. ATL,-10.5,10.5,0.26,0.74,...,6,4,2,0,1,2,5,0,0,5
37,2012,0021200041,2012-11-04,47,65,LAL vs. DET,-8.5,8.5,0.30,0.70,...,3,1,0,0,0,0,6,0,1,2
38,2012,0021200038,2012-11-04,61,50,TOR vs. MIN,-3.5,3.5,0.41,0.59,...,4,2,2,0,1,3,6,4,4,1
39,2012,0021200039,2012-11-04,53,56,ORL vs. PHX,-1.5,-1.5,0.46,0.46,...,1,2,2,0,0,1,4,2,2,1
